In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchvision import models
from torchvision.datasets import ImageFolder
from torch.utils.data import random_split

BATCH_SIZE = 32
NUM_EPOCHS = 10
LEARNING_RATE = 0.001
VAL_SPLIT = 0.2
IMG_SIZE = (224, 224)
NORMALIZE_MEAN = [0.485, 0.456, 0.406]
NORMALIZE_STD = [0.229, 0.224, 0.225]
NUM_WORKERS = 2
DATA_DIR = "/Users/griffosx/Projects/fda/assets/train"
SAVE_PATH = "/Users/griffosx/Projects/fda/classifier.pth"

# Add path
import sys
from pathlib import Path

sys.path.append(str(Path().absolute().parent))

from clustering.classification_model import get_model, Trainer

In [ ]:
if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print(f"Using device: {device}")

# Define transforms
transform = transforms.Compose(
    [
        transforms.Resize(IMG_SIZE),
        transforms.ToTensor(),
        # transforms.Normalize(mean=NORMALIZE_MEAN, std=NORMALIZE_STD),
    ]
)

# Create dataset using ImageFolder (handles class folders automatically)
full_dataset = ImageFolder(root=DATA_DIR, transform=transform)

# Get class count and mapping
num_classes = len(full_dataset.classes)
print(f"Number of classes: {num_classes}")
print(f"Classes: {full_dataset.classes}")

Using device: mps
Number of classes: 5
Classes: ['bear', 'bull', 'crazy', 'mid_bear', 'mid_bull']


In [ ]:
# Split dataset into train and validation
dataset_size = len(full_dataset)
val_size = int(VAL_SPLIT * dataset_size)
train_size = dataset_size - val_size

train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

# Create data loaders
train_loader = DataLoader(
    train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS
)
val_loader = DataLoader(
    val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS
)


In [ ]:

# Initialize model
model = get_model(num_classes=num_classes, use_pretrained=True)
model = model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

# Create trainer and train
trainer = Trainer(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    criterion=criterion,
    optimizer=optimizer,
    device=device,
)

trainer.train(num_epochs=NUM_EPOCHS, save_path=SAVE_PATH)

print("Training completed!")

Epoch 1/10:
Train Loss: 0.8193 | Train Acc: 70.81%
Val Loss: 0.6742 | Val Acc: 81.01%
Model saved to /Users/griffosx/Projects/fda/classified.pth
--------------------------------------------------
Epoch 2/10:
Train Loss: 0.4305 | Train Acc: 83.66%
Val Loss: 0.4690 | Val Acc: 86.03%
Model saved to /Users/griffosx/Projects/fda/classified.pth
--------------------------------------------------
Epoch 3/10:
Train Loss: 0.3312 | Train Acc: 87.71%
Val Loss: 0.4165 | Val Acc: 83.24%
--------------------------------------------------
Epoch 4/10:
Train Loss: 0.2526 | Train Acc: 91.48%
Val Loss: 0.3781 | Val Acc: 84.92%
--------------------------------------------------
Epoch 5/10:
Train Loss: 0.3203 | Train Acc: 89.25%
Val Loss: 1.0968 | Val Acc: 78.21%
--------------------------------------------------
Epoch 6/10:
Train Loss: 0.3155 | Train Acc: 89.11%
Val Loss: 0.3631 | Val Acc: 89.94%
Model saved to /Users/griffosx/Projects/fda/classified.pth
--------------------------------------------------
E